In [0]:
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import os

Using TensorFlow backend.


In [0]:
path = keras.utils.get_file("machine.data", "https://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data")
path

16384/8726 [========================================================] - 0s 0us/step


'/root/.keras/datasets/machine.data'

In [0]:
column_names =['vendor','Model_Name','MYCT','MMIN','MMAX','CACH','CHMIN','CHMAX','PRP','ERP']
dataset = pd.read_csv(path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=True)

dataset.head(5)

vendor Model_Name  MYCT  MMIN   MMAX  CACH  CHMIN  CHMAX  PRP  ERP
0  adviser      32/60   125   256   6000   256     16    128  198  199
1   amdahl     470v/7    29  8000  32000    32      8     32  269  253
2   amdahl    470v/7a    29  8000  32000    32      8     32  220  253
3   amdahl    470v/7b    29  8000  32000    32      8     32  172  253
4   amdahl    470v/7c    29  8000  16000    32      8     16  132  132

In [0]:
dataset=dataset.drop(columns=['vendor','Model_Name'])


In [0]:
y=dataset.pop("ERP")
X=dataset.copy()

In [0]:
X_stats = X.describe()
X_stats= X_stats.transpose()
X_stats

count          mean           std  ...     50%      75%      max
MYCT   209.0    203.822967    260.262926  ...   110.0    225.0   1500.0
MMIN   209.0   2867.980861   3878.742758  ...  2000.0   4000.0  32000.0
MMAX   209.0  11796.153110  11726.564377  ...  8000.0  16000.0  64000.0
CACH   209.0     25.205742     40.628722  ...     8.0     32.0    256.0
CHMIN  209.0      4.698565      6.816274  ...     2.0      6.0     52.0
CHMAX  209.0     18.267943     25.997318  ...     8.0     24.0    176.0
PRP    209.0    105.622010    160.830733  ...    50.0    113.0   1150.0

[7 rows x 8 columns]

In [0]:
X_normalized = (X - X_stats['mean']) / X_stats['std']
X_normalized

MYCT      MMIN      MMAX      CACH     CHMIN     CHMAX       PRP
0   -0.302859 -0.673409 -0.494275  5.680569  1.658008  4.220899  0.574380
1   -0.671717  1.323114  1.722913  0.167228  0.484346  0.528211  1.015838
2   -0.671717  1.323114  1.722913  0.167228  0.484346  0.528211  0.711170
3   -0.671717  1.323114  1.722913  0.167228  0.484346  0.528211  0.412720
4   -0.671717  1.323114  0.358489  0.167228  0.484346 -0.087238  0.164011
..        ...       ...       ...       ...       ...       ...       ...
204 -0.306701 -0.481594 -0.323723 -0.620392 -0.542608 -0.394962 -0.395584
205 -0.406600 -0.481594 -0.323723  0.167228 -0.395900 -0.394962 -0.370713
206 -0.302859 -0.223779 -0.323723 -0.620392 -0.395900 -0.164169 -0.333406
207  1.061146 -0.607408 -0.323723  0.167228 -0.689316 -0.702686 -0.240141
208  1.061146 -0.481594 -0.664828 -0.620392 -0.689316 -0.702686 -0.376931

[209 rows x 7 columns]

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=[len(X_normalized.keys())]),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

In [0]:
model.compile(loss='mse',
                optimizer='adam',
                metrics=['mse'])

In [0]:
model.fit(X_normalized,y,epochs=1000,validation_split=0.1)


Epoch 1/1000
6/6 [==============================] - 0s 26ms/step - loss: 24538.9609 - mse: 24538.9609 - val_loss: 115272.5156 - val_mse: 115272.5156
Epoch 2/1000
6/6 [==============================] - 0s 6ms/step - loss: 24404.0371 - mse: 24404.0371 - val_loss: 114643.9531 - val_mse: 114643.9531
Epoch 3/1000
6/6 [==============================] - 0s 6ms/step - loss: 24266.9102 - mse: 24266.9102 - val_loss: 113978.3125 - val_mse: 113978.3125
Epoch 4/1000
6/6 [==============================] - 0s 6ms/step - loss: 24098.0488 - mse: 24098.0488 - val_loss: 113162.2891 - val_mse: 113162.2891
Epoch 5/1000
6/6 [==============================] - 0s 6ms/step - loss: 23894.5723 - mse: 23894.5723 - val_loss: 112160.3438 - val_mse: 112160.3438
Epoch 6/1000
6/6 [==============================] - 0s 5ms/step - loss: 23645.9141 - mse: 23645.9141 - val_loss: 110871.0156 - val_mse: 110871.0156
Epoch 7/1000
6/6 [==============================] - 0s 6ms/step - loss: 23298.8770 - mse: 23298.8770 - val_loss